## Adaptive RAG Agent with Langgraph
 - Follows ideas from 3 papers:

    1. Adaptive RAG
    2. Corrective RAG
    3. Self Correction in RAG

In [1]:
# ollama pull llama3.2:3b-instruct-fp16

In [2]:
## LLM
from langchain_ollama import ChatOllama

local_llm = "llama3.2:3b-instruct-fp16"
llm = ChatOllama(name="llama", model=local_llm, request_timeout=120.0)
llm_json_mode = ChatOllama(name="llama", model=local_llm, request_timeout=120.0, json_mode=True)